In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json



In [2]:
# Get Statistic Area level 2 information from csv file
sa2_df = pd.read_csv("resources/SA2_2016_AUST.csv")

# Filter the information to show Greater Perth Only
sa2_perth_df = sa2_df[sa2_df["GCCSA_NAME_2016"] == "Greater Perth"]

sa2_perth_df.head()


,SA2_MAINCODE_2016,SA2_5DIGITCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,GCCSA_NAME_2016,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
1768,502011021,51021,Dawesville - Bouvard,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,97.9248
1769,502011022,51022,Falcon - Wannanup,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,9.5893
1770,502011023,51023,Greenfields,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,9.9609
1771,502011024,51024,Halls Head - Erskine,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,14.2469
1772,502011025,51025,Mandurah,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,8.3954


In [5]:
# Store information into the dictionaly
rent_dict = {"SA2 Name":[],"$1-$74":[], "$75-$99":[], "$100-$124":[], "$125-$149":[],
              "$150-$174":[], "$175-$199":[], "$200-$224":[], "$225-$249":[], "$250-$274":[],
              "$275-$299":[], "$300-$324":[], "$325-$349":[], "$350-$374":[], "$375-$399":[],
              "$400-$424":[], "$425-$449":[], "$450-$549":[], "$550-$649":[], "$650-$749":[],
              "$750-$849":[], "$850-$949":[], "$950 and over":[]}

#   Data:  One family household with only family members present
for code in sa2_perth_df["SA2_MAINCODE_2016"]:
    sa2_code = str(code)
    report_range = np.arange(2,24)
    rent_dict["SA2 Name"].append(sa2_perth_df.loc[sa2_perth_df["SA2_MAINCODE_2016"]==code,"SA2_NAME_2016"].values[0])
# Report type start with 03 and finish 22
    for number in report_range:
        report_type = str(number)
        abs_url = f"http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T26_SA/TOT.{report_type}.5.SA2.{code}/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2016"
        response = requests.get(abs_url)
        data = response.json()
#abs data name is set as key of dictionary, and data was included in observations
        rent_dict[f'{data["structure"]["dimensions"]["observation"][1]["values"][0]["name"]}'].append(data["dataSets"][0]["observations"]["0:0:0:0:0:0"][0])




In [8]:
rent_df = pd.DataFrame(rent_dict)

In [9]:
# Rename the columns to see the data more easily
rent_df.columns = ["SA2 Name", "$1-$74", "$75-$99", "$100-$124", "$125-$149",
                   "$150-$174", "$175-$199", "$200-$224", "$225-$249", "$250-$274",
                   "$275-$299", "$300-$324", "$325-$349", "$350-$374", "$375-$399",
                   "$400-$424", "$425-$449", "$450-$549", "$550-$649", "$650-$749",
                   "$750-$849", "$850-$949", "$950 and over"]

# To calculate the median from histogram 
bin_size = [75, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 100, 100, 100, 100, 100, 100]
bin_left = [1, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 550, 650, 750, 850, 950]

# Median from Histogram

Use interpolated Median


reference url
https://www.vivaxsolutions.com/maths/allnrintpltn.aspx

https://math.stackexchange.com/questions/2591946/how-to-find-median-from-a-histogram

Median=   l+  (n/2−F)/fm *w
where l is the lower border of the median group, F is the cumulative frequency up to the median group, fm is the frequency of the median group, w is the width of the median group. 

In [10]:
rent_df["Total"] = rent_df.iloc[:,1:23].sum(axis=1)

rent_df

,SA2 Name,$1-$74,$75-$99,$100-$124,$125-$149,$150-$174,$175-$199,$200-$224,$225-$249,$250-$274,...,$375-$399,$400-$424,$425-$449,$450-$549,$550-$649,$650-$749,$750-$849,$850-$949,$950 and over,Total
0,Dawesville - Bouvard,0.0,3.0,6.0,0.0,18.0,10.0,30.0,15.0,21.0,...,37.0,50.0,6.0,58.0,17.0,4.0,0.0,0.0,3.0,437.0
1,Falcon - Wannanup,0.0,0.0,3.0,9.0,22.0,16.0,43.0,23.0,71.0,...,59.0,83.0,26.0,71.0,14.0,3.0,0.0,0.0,3.0,809.0
2,Greenfields,6.0,15.0,48.0,13.0,41.0,25.0,46.0,28.0,79.0,...,34.0,14.0,7.0,7.0,0.0,3.0,0.0,0.0,0.0,859.0
3,Halls Head - Erskine,3.0,6.0,18.0,12.0,31.0,15.0,32.0,32.0,82.0,...,139.0,198.0,58.0,146.0,21.0,9.0,0.0,0.0,3.0,1438.0
4,Mandurah,13.0,23.0,115.0,48.0,94.0,72.0,112.0,126.0,237.0,...,43.0,60.0,25.0,80.0,27.0,7.0,4.0,0.0,11.0,1948.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Rockingham Lakes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169,Safety Bay - Shoalwater,11.0,10.0,48.0,16.0,21.0,13.0,66.0,68.0,138.0,...,82.0,91.0,32.0,50.0,9.0,3.0,0.0,0.0,3.0,1169.0
170,Singleton - Golden Bay - Secret Harbour,5.0,0.0,4.0,7.0,9.0,4.0,12.0,19.0,53.0,...,164.0,231.0,70.0,212.0,47.0,10.0,5.0,3.0,0.0,1273.0
171,Waikiki,12.0,10.0,32.0,15.0,37.0,14.0,25.0,31.0,55.0,...,127.0,105.0,41.0,54.0,3.0,0.0,0.0,0.0,3.0,1007.0


In [11]:
clean_df = rent_df[rent_df["Total"] > 100]

In [12]:
clean_df["Median"] =""
clean_df = clean_df.reset_index()
clean_df = clean_df.iloc[:,1:]

clean_df.columns

C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Index(['SA2 Name', '$1-$74', '$75-$99', '$100-$124', '$125-$149', '$150-$174',
       '$175-$199', '$200-$224', '$225-$249', '$250-$274', '$275-$299',
       '$300-$324', '$325-$349', '$350-$374', '$375-$399', '$400-$424',
       '$425-$449', '$450-$549', '$550-$649', '$650-$749', '$750-$849',
       '$850-$949', '$950 and over', 'Total', 'Median'],
      dtype='object')

In [13]:
#Calculate Median from histgram value (search the index which exceeds (total number /2) )
for index_k,row in clean_df.iterrows():
    counter = 0
    i = 1
    while counter < (row[23]+1)/2:
        counter = counter + row[i]
        i = i + 1
# Linear interpolation   lower border + ((total number +1)/2 - frequency up to the median group) / frequency of median group  * width of median group
    median = bin_left[i-2] + ((row[23]+1)/2 - (counter - row[i-1])) * bin_size[i-2] / (row[i-1])
    clean_df.iloc[index_k,24] = median
    

In [15]:
clean_df

,SA2 Name,$1-$74,$75-$99,$100-$124,$125-$149,$150-$174,$175-$199,$200-$224,$225-$249,$250-$274,...,$400-$424,$425-$449,$450-$549,$550-$649,$650-$749,$750-$849,$850-$949,$950 and over,Total,Median
0,Dawesville - Bouvard,0.0,3.0,6.0,0.0,18.0,10.0,30.0,15.0,21.0,...,50.0,6.0,58.0,17.0,4.0,0.0,0.0,3.0,437.0,358.955
1,Falcon - Wannanup,0.0,0.0,3.0,9.0,22.0,16.0,43.0,23.0,71.0,...,83.0,26.0,71.0,14.0,3.0,0.0,0.0,3.0,809.0,342.46
2,Greenfields,6.0,15.0,48.0,13.0,41.0,25.0,46.0,28.0,79.0,...,14.0,7.0,7.0,0.0,3.0,0.0,0.0,0.0,859.0,306.12
3,Halls Head - Erskine,3.0,6.0,18.0,12.0,31.0,15.0,32.0,32.0,82.0,...,198.0,58.0,146.0,21.0,9.0,0.0,0.0,3.0,1438.0,360.374
4,Mandurah,13.0,23.0,115.0,48.0,94.0,72.0,112.0,126.0,237.0,...,60.0,25.0,80.0,27.0,7.0,4.0,0.0,11.0,1948.0,290.354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Rockingham,16.0,22.0,88.0,44.0,99.0,47.0,91.0,95.0,311.0,...,173.0,55.0,90.0,22.0,6.0,4.0,0.0,11.0,2145.0,305.314
144,Safety Bay - Shoalwater,11.0,10.0,48.0,16.0,21.0,13.0,66.0,68.0,138.0,...,91.0,32.0,50.0,9.0,3.0,0.0,0.0,3.0,1169.0,314.106
145,Singleton - Golden Bay - Secret Harbour,5.0,0.0,4.0,7.0,9.0,4.0,12.0,19.0,53.0,...,231.0,70.0,212.0,47.0,10.0,5.0,3.0,0.0,1273.0,391.159
146,Waikiki,12.0,10.0,32.0,15.0,37.0,14.0,25.0,31.0,55.0,...,105.0,41.0,54.0,3.0,0.0,0.0,0.0,3.0,1007.0,350.291


In [16]:
clean_df.to_csv("resources/rent.csv")

In [17]:
# get suburb name (SA2) and Median
clean_df[["SA2 Name","Median"]]

,SA2 Name,Median
0,Dawesville - Bouvard,358.955
1,Falcon - Wannanup,342.46
2,Greenfields,306.12
3,Halls Head - Erskine,360.374
4,Mandurah,290.354
...,...,...
143,Rockingham,305.314
144,Safety Bay - Shoalwater,314.106
145,Singleton - Golden Bay - Secret Harbour,391.159
146,Waikiki,350.291


In [18]:
repay_median = clean_df[["SA2 Name","Median"]]

In [19]:
repay_median.to_csv("resources/rent_median.csv")